In [9]:
import json
import os
import nest_asyncio

nest_asyncio.apply()

def get_config_data():
    with open("local_data.json") as json_file:
        data = json.load(json_file)
        return data["OPENAI_KEY"]["key"]
        
OPENAI_API_KEY = get_config_data()

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import QueryEngineTool, ToolMetadata

# llm = OpenAI(model="gpt-3.5-turbo")
llm = OpenAI(model="gpt-4o-mini")



In [18]:
## Define tools

# Tool to accept user input to retrieve data from GEO using existing functions and ARCHS4PY
from llama_index.core.tools import FunctionTool
from pydantic import Field


def get_weather(
    location: str = Field(
        description="A city name and state, formatted like '<name>, <state>'"
    ),
) -> str:
    """Useful for getting the weather for a given location."""
    return f"The weather in {location} is sunny."


tool = FunctionTool.from_defaults(get_weather)
tools = [tool]

agent = ReActAgent.from_tools(tools, llm=llm, verbose=True)

response = agent.chat("Miami, FL")

print(response)


> Running step fbabf731-f6fb-4e35-a86f-0662a41fe8a1. Step input: Miami, FL
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: get_weather
Action Input: {'location': 'Miami, FL'}
Observation: The weather in Miami, FL is sunny.
> Running step b58d303a-f8e1-482a-a1d5-2f5c453e490c. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The weather in Miami, FL is sunny.
The weather in Miami, FL is sunny.


In [ ]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI

# import and define tools
...

# initialize llm
llm = OpenAI(model="gpt-4o-mini")

# initialize openai agent
agent = OpenAIAgent.from_tools(tools, llm=llm, verbose=True)

#from llama_index.core.agent import AgentRunner, ReActAgent
#from llama_index.agent.openai import OpenAIAgentWorker, OpenAIAgent
#from llama_index.agent.openai import OpenAIAgentWorker

#agent_llm = OpenAI(model="gpt-4o-mini")
# agent_llm = OpenAI(model="gpt-4-1106-preview")

#agent = ReActAgent.from_tools(
#    tools, llm=agent_llm, verbose=True, max_iterations=20
#)

In [ ]:

### Old agent tool example

march_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_march_2022.pdf"]
).load_data()
june_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_june_2022.pdf"]
).load_data()
sept_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_sept_2022.pdf"]
).load_data()

def get_tool(name, full_name, documents=None):
    if not os.path.exists(f"./data/{name}"):
        # build vector index
        vector_index = VectorStoreIndex.from_documents(documents)
        vector_index.storage_context.persist(persist_dir=f"./data/{name}")
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./data/{name}"),
        )
    query_engine = vector_index.as_query_engine(similarity_top_k=3, llm=llm)
    query_engine_tool = QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name=name,
            description=(
                "Provides information about Uber quarterly financials ending"
                f" {full_name}"
            ),
        ),
    )
    return query_engine_tool

march_tool = get_tool("march_2022", "March 2022", documents=march_2022)
june_tool = get_tool("june_2022", "June 2022", documents=june_2022)
sept_tool = get_tool("sept_2022", "September 2022", documents=sept_2022)

query_engine_tools = [march_tool, june_tool, sept_tool]


In [ ]:
## Create human in the loop agent with tools

from llama_index.core.agent import AgentRunner, ReActAgent
from llama_index.agent.openai import OpenAIAgentWorker, OpenAIAgent
from llama_index.agent.openai import OpenAIAgentWorker

agent_llm = OpenAI(model="gpt-4o-mini")
# agent_llm = OpenAI(model="gpt-4-1106-preview")

agent = ReActAgent.from_tools(
    query_engine_tools, llm=agent_llm, verbose=True, max_iterations=20
)

def chat_repl(exit_when_done: bool = True):
    """Chat REPL.

    Args:
        exit_when_done(bool): if True, automatically exit when step is finished.
            Set to False if you want to keep going even if step is marked as finished by the agent.
            If False, you need to explicitly call "exit" to finalize a task execution.

    """
    task_message = None
    while task_message != "exit":
        task_message = input(">> Human: ")
        if task_message == "exit":
            break

        task = agent.create_task(task_message)

        response = None
        step_output = None
        message = None
        while message != "exit":
            if message is None or message == "":
                step_output = agent.run_step(task.task_id)
            else:
                step_output = agent.run_step(task.task_id, input=message)
            if exit_when_done and step_output.is_last:
                print(
                    ">> Task marked as finished by the agent, executing task execution."
                )
                break

            message = input(
                ">> Add feedback during step? (press enter/leave blank to continue, and type 'exit' to stop): "
            )
            if message == "exit":
                break

        if step_output is None:
            print(">> You haven't run the agent. Task is discarded.")
        elif not step_output.is_last:
            print(">> The agent hasn't finished yet. Task is discarded.")
        else:
            response = agent.finalize_response(task.task_id)
        print(f"Agent: {str(response)}")

In [ ]:
## Test agent
chat_repl()